<h1>Neural Network SMS Text Classifier ML </h1>

In [1]:
# import libraries

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


print(tf.__version__)

2024-04-12 21:45:34.712185: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 21:45:35.383978: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


/home/marius/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.16.1


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from textattack.augmentation import EmbeddingAugmenter
import re

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-04-12 21:45:41--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::681a:321, 2606:4700:20::681a:221, 2606:4700:20::ac43:4695, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::681a:321|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.10’

train-data.tsv.10   100%[===================>] 349,84K  --.-KB/s    in 0,01s   

2024-04-12 21:45:41 (29,6 MB/s) - ‘train-data.tsv.10’ saved [358233/358233]

--2024-04-12 21:45:41--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::ac43:4695, 2606:4700:20::681a:221, 2606:4700:20::681a:321, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::ac43:4695|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [te

In [4]:
# Make dataframe of dataset to see data

dataset_train = pd.read_csv(train_file_path, sep='\t')

# Rename the first column to 'label' and second column to 'text' and reverse the order
list_columns = dataset_train.columns.tolist()
list_columns[1] = 'text'
list_columns[0] = 'label'
dataset_train.columns = list_columns

dataset_train = dataset_train.iloc[:, ::-1]
dataset_train.head(10)

,text,label
0,you can never do nothing,ham
1,"now u sound like manky scouse boy steve,like! ...",ham
2,mum say we wan to go then go... then she can s...,ham
3,never y lei... i v lazy... got wat? dat day ü ...,ham
4,in xam hall boy asked girl tell me the startin...,ham
5,genius what's up. how your brother. pls send h...,ham
6,they finally came to fix the ceiling.,ham
7,urgent! call 09066350750 from your landline. y...,spam
8,now that you have started dont stop. just pray...,ham
9,hello darlin ive finished college now so txt m...,ham


In [5]:
dataset_test = pd.read_csv(test_file_path, sep='\t')

# Rename the first column to 'label' and second column to 'text' and reverse the order
list_columns = dataset_test.columns.tolist()
list_columns[1] = 'text'
list_columns[0] = 'label'
dataset_test.columns = list_columns

dataset_test = dataset_test.iloc[:, ::-1]
dataset_test.head(10)

,text,label
0,"not much, just some textin'. how bout you?",ham
1,i probably won't eat at all today. i think i'm...,ham
2,don‘t give a flying monkeys wot they think and...,ham
3,who are you seeing?,ham
4,your opinion about me? 1. over 2. jada 3. kusr...,ham
5,yesterday its with me only . now am going home.,ham
6,yes. it's all innocent fun. o:-),ham
7,a boy was late 2 home. his father: power of fr...,ham
8,is ur changes 2 da report big? cos i've alread...,ham
9,sorry about earlier. putting out fires.are you...,ham


In [6]:
# Split messages and labels from train data

texts, labels = dataset_train['text'], dataset_train['label']
texts

0                                you can never do nothing
1       now u sound like manky scouse boy steve,like! ...
2       mum say we wan to go then go... then she can s...
3       never y lei... i v lazy... got wat? dat day ü ...
4       in xam hall boy asked girl tell me the startin...
                              ...                        
4173    just woke up. yeesh its late. but i didn't fal...
4174    what do u reckon as need 2 arrange transport i...
4175    free entry into our £250 weekly competition ju...
4176    -pls stop bootydelious (32/f) is inviting you ...
4177    tell my  bad character which u dnt lik in me. ...
Name: text, Length: 4178, dtype: object

In [7]:
labels

0        ham
1        ham
2        ham
3        ham
4        ham
        ... 
4173     ham
4174     ham
4175    spam
4176    spam
4177     ham
Name: label, Length: 4178, dtype: object

In [8]:
# Define regular expressions for emails, websites, and phone numbers

email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
website_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
phone_number_regex = r'\b(?:\+\d{1,2}\s)?\d{3}[-.\s]?\d{3}[-.\s]?\d{4}\b'


In [9]:
# Spliting data into data and test data

X_train = dataset_train['text']
y_train = dataset_train['label']
X_test = dataset_test['text']
y_test = dataset_test['label']

In [10]:
# Identify and replace emails, websites, and phone numbers with special tokens
X_train_cleaned = X_train.str.replace(email_regex, 'email')
X_train_cleaned = X_train_cleaned.str.replace(website_regex, 'website')
X_train_cleaned = X_train_cleaned.str.replace(phone_number_regex, 'phonenumber')

X_test_cleaned = X_test.str.replace(email_regex, 'email')
X_test_cleaned = X_test_cleaned.str.replace(website_regex, 'website')
X_test_cleaned = X_test_cleaned.str.replace(phone_number_regex, 'phonenumber')

In [11]:
# Convert labels from text to numerical

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_train_encoded

array([0, 0, 0, ..., 1, 1, 0])

In [12]:
y_test_encoded

array([0, 0, 0, ..., 0, 1, 1])

In [13]:
# Tokenize the text

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_cleaned)

In [14]:
# Convert text to sequences (numerical)

X_train_seq = tokenizer.texts_to_sequences(X_train_cleaned)
X_test_seq = tokenizer.texts_to_sequences(X_test_cleaned)

In [15]:
X_train_seq

[[3, 29, 281, 27, 340],
 [19,
  6,
  830,
  55,
  3665,
  3666,
  357,
  3667,
  55,
  1,
  9,
  3668,
  18,
  94,
  414,
  85,
  493,
  113,
  6,
  3669,
  41,
  3670,
  434,
  537],
 [765,
  144,
  40,
  276,
  2,
  46,
  56,
  46,
  56,
  88,
  29,
  3671,
  3672,
  341,
  94,
  3673,
  3674],
 [281,
  314,
  606,
  1,
  261,
  971,
  58,
  145,
  315,
  62,
  77,
  68,
  10,
  94,
  1615,
  197,
  140,
  75],
 [8,
  3675,
  3676,
  357,
  515,
  374,
  99,
  10,
  5,
  1204,
  1943,
  12,
  434,
  516,
  1,
  29,
  375,
  2482,
  18,
  11,
  972,
  157,
  301,
  15,
  3677,
  82,
  1205,
  212,
  2483,
  88,
  159,
  5,
  3678,
  3679,
  3680],
 [2484, 538, 44, 49, 13, 539, 97, 68, 203, 147, 2, 11, 1616],
 [107, 725, 415, 2, 1381, 5, 3681],
 [217,
  16,
  2485,
  52,
  13,
  401,
  13,
  890,
  41,
  1944,
  271,
  26,
  435,
  517,
  160,
  576,
  460,
  577,
  162,
  316,
  376,
  327,
  1617,
  2486,
  2487,
  973,
  1945,
  272],
 [19,
  20,
  3,
  17,
  518,
  95,
  87,
  36,


In [16]:
X_test_seq

[[25, 115, 36, 119, 49, 809, 3],
 [1,
  381,
  558,
  365,
  28,
  51,
  90,
  1,
  110,
  30,
  199,
  6647,
  49,
  65,
  13,
  419,
  118,
  6,
  172,
  10],
 [3537,
  150,
  4,
  493,
  107,
  110,
  7,
  1,
  6336,
  3537,
  308,
  109,
  246,
  15,
  852,
  7,
  51,
  20],
 [127, 22, 3, 864],
 [13,
  1399,
  79,
  10,
  131,
  244,
  21,
  3653,
  165,
  3654,
  41,
  895,
  225,
  1104,
  289,
  2466,
  1723,
  304,
  25,
  3655,
  439,
  3656,
  410,
  724,
  97,
  92],
 [527, 60, 33, 10, 63, 19, 64, 74, 85],
 [133, 138, 51, 5876, 507, 719],
 [4, 357, 65, 219, 21, 85, 203, 878, 2162, 15, 2076],
 [9,
  37,
  7668,
  21,
  94,
  2032,
  352,
  166,
  200,
  156,
  405,
  7668,
  21,
  94,
  3346,
  2032],
 [91,
  79,
  822,
  2452,
  50,
  22,
  3,
  212,
  2,
  295,
  157,
  410,
  26,
  27,
  3,
  400,
  17,
  4,
  169,
  213],
 [30, 80, 486, 12, 4, 286, 2, 240, 7, 17, 25, 697, 522, 602, 280],
 [97, 68, 10, 13, 594, 395],
 [118, 6, 58, 20, 2204, 692],
 [1, 67, 2, 2861, 1, 73, 2

In [17]:
# Pad sequences to ensure uniform length

max_sequence_length = max([len(seq) for seq in X_train_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_sequence_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_sequence_length, padding='post')


In [18]:
X_train_padded

array([[   3,   29,  281, ...,    0,    0,    0],
       [  19,    6,  830, ...,    0,    0,    0],
       [ 765,  144,   40, ...,    0,    0,    0],
       ...,
       [  47,  463,  337, ...,    0,    0,    0],
       [  97,   87, 3664, ...,    0,    0,    0],
       [  99,   11,  432, ...,    0,    0,    0]], dtype=int32)

In [19]:
X_test_seq

[[25, 115, 36, 119, 49, 809, 3],
 [1,
  381,
  558,
  365,
  28,
  51,
  90,
  1,
  110,
  30,
  199,
  6647,
  49,
  65,
  13,
  419,
  118,
  6,
  172,
  10],
 [3537,
  150,
  4,
  493,
  107,
  110,
  7,
  1,
  6336,
  3537,
  308,
  109,
  246,
  15,
  852,
  7,
  51,
  20],
 [127, 22, 3, 864],
 [13,
  1399,
  79,
  10,
  131,
  244,
  21,
  3653,
  165,
  3654,
  41,
  895,
  225,
  1104,
  289,
  2466,
  1723,
  304,
  25,
  3655,
  439,
  3656,
  410,
  724,
  97,
  92],
 [527, 60, 33, 10, 63, 19, 64, 74, 85],
 [133, 138, 51, 5876, 507, 719],
 [4, 357, 65, 219, 21, 85, 203, 878, 2162, 15, 2076],
 [9,
  37,
  7668,
  21,
  94,
  2032,
  352,
  166,
  200,
  156,
  405,
  7668,
  21,
  94,
  3346,
  2032],
 [91,
  79,
  822,
  2452,
  50,
  22,
  3,
  212,
  2,
  295,
  157,
  410,
  26,
  27,
  3,
  400,
  17,
  4,
  169,
  213],
 [30, 80, 486, 12, 4, 286, 2, 240, 7, 17, 25, 697, 522, 602, 280],
 [97, 68, 10, 13, 594, 395],
 [118, 6, 58, 20, 2204, 692],
 [1, 67, 2, 2861, 1, 73, 2

In [20]:
# Build the model with additional features

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_length=max_sequence_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout for regularization
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout for regularization
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [21]:
# Compile the model with ADAM optimizer

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])



In [22]:
# Train the model

model.fit(X_train_padded, y_train_encoded, epochs=10, batch_size=64, validation_split=0.2)


Epoch 1/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8058 - loss: 0.5008 - val_accuracy: 0.8612 - val_loss: 0.4335
Epoch 2/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8708 - loss: 0.3934 - val_accuracy: 0.8612 - val_loss: 0.4301
Epoch 3/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8746 - loss: 0.3769 - val_accuracy: 0.8612 - val_loss: 0.4283
Epoch 4/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8734 - loss: 0.3815 - val_accuracy: 0.8612 - val_loss: 0.4102
Epoch 5/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8644 - loss: 0.3796 - val_accuracy: 0.8612 - val_loss: 0.3886
Epoch 6/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8615 - loss: 0.3461 - val_accuracy: 0.8612 - val_loss: 0.3373
Epoch 7/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8881 - loss: 0.2464 - val_accuracy: 0.8708 - val_loss: 0.3753
Epoch 8/10
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9250 - loss: 0.2463 - val_accuracy: 0.9378 - v

In [23]:
# Evaluate the model

test_loss, test_accuracy = model.evaluate(X_test_padded, y_test_encoded)

44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9800 - loss: 0.0663


In [24]:
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.9777138829231262


In [25]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    # Preprocess the input text
    pred_text_cleaned = re.sub(email_regex, 'email', pred_text)
    pred_text_cleaned = re.sub(website_regex, 'website', pred_text_cleaned)
    pred_text_cleaned = re.sub(phone_number_regex, 'phonenumber', pred_text_cleaned)
    
    pred_seq = tokenizer.texts_to_sequences([pred_text_cleaned])
    pred_padded = pad_sequences(pred_seq, maxlen=max_sequence_length, padding='post')
    # Predict
    prediction = model.predict(pred_padded)[0][0]
    label = "spam" if prediction > 0.5 else "ham"
    return [prediction, label]


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
[0.008892687, 'ham']


In [34]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
    test_messages = ["how are you doing today",
                    "sale today! to stop texts call 98912460324",
                    "i dont want to go. can we try it a different day? available sat",
                    "our new mobile video service is live. just install on your phone to start watching.",
                    "you have won £1000 cash! call to claim your prize.",
                    "i'll bring it tomorrow. don't forget the milk.",
                    "wow, is your arm alright. that happened to me one time too"
                    ]
    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        print("Message:", msg)
        print("Prediction:", prediction[1])
        print("True Label:", ans)
        if prediction[1] != ans:
            passed = False
            
    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

# Run the test
test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Message: how are you doing today
Prediction: ham
True Label: ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Message: sale today! to stop texts call 98912460324
Prediction: spam
True Label: spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Message: i dont want to go. can we try it a different day? available sat
Prediction: ham
True Label: ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Message: our new mobile video service is live. just install on your phone to start watching.
Prediction: spam
True Label: spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Message: you have won £1000 cash! call to claim your prize.
Prediction: spam
True Label: spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Message: i'll bring it tomorrow. don't forget the milk.
Prediction: ham
True Label: ham
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Message: wow, is your arm alright. that happened to me one time too
Prediction: ham
True Label: ham
You passed the challenge. Great job!
